In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utility import *
from theoretical import *
from scipy.stats import sem, norm, bernoulli

# Force scientific notation
pd.set_option("display.float_format", "{:.2e}".format)

## Chapter 7 - Importance Sampling

### Basic Ideas of Importance Sampling

**Example 7.1.** Assume that under the risk-neutral probability measure, the price of the underlying stock is a geometric Brownian motion
$$ S_t = S_0 \exp \left\{ \left( r - \frac{1}{2} \sigma^2 \right) t + \sigma W_t\right\}. $$
Use importance sampling to estimate the price of a binary call option with maturity $T$ and payoff
$$ 1_{\{S_T \geq K\}.} $$


In [2]:
def example_7_1():
    
    np.random.seed(4198)
    
    # simulation params
    S_0, r, sig, T = 50, 0.01, 0.1, 1
    n = 10_000
    
    def importance_sampling(K):
        b = (np.log(K / S_0) - (r - sig**2 * 0.5) * T) / (sig * np.sqrt(T))
        
        x_star = np.max(b, 0)

        Y = np.random.normal(loc=x_star, size=n)
        H = np.exp(-r * T - x_star * Y + x_star**2 * 0.5) * (Y >= b)

        # compute mean est. and std. error
        est = np.mean(H)
        se = sem(H)
        re = se / np.abs(est)

        return est, se, re
    
    def plain_sampling(K):
        # sampling
        Z = np.random.normal(size=n)
        Y = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * Z)
        X = np.exp(-r * T) * (Y >= K)

        # compute mean est. and std. error
        est = np.mean(X)
        se = sem(X)
        re = se / np.abs(est)

        return est, se, re

    theoretical = {}
    mc_est = {}
    se_vals = {}
    re_vals = {}
    strikes, methods = [50, 60, 70, 80], ["IS", "plain"]
    
    for k in strikes:
        theoretical_val = binary_call(S_0, k, r, T, sig) 
        mc_est_plain, se_plain, re_plain = plain_sampling(k)
        mc_est_importance, se_importance, re_importance = importance_sampling(k)

        importance_key = (f"{k}", "IS")
        plain_key = (f"{k}", "plain")

        theoretical[importance_key] = theoretical_val
        mc_est[importance_key] = mc_est_importance
        se_vals[importance_key] = se_importance
        re_vals[importance_key] = re_importance

        theoretical[plain_key] = ""
        mc_est[plain_key] = mc_est_plain
        se_vals[plain_key] = se_plain
        re_vals[plain_key] = re_plain
    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [strikes, methods],
        names=["Strike price", ""]
    )

    # Collect rows
    rows = {
        "True value": [theoretical[(str(k), m)] for k in strikes for m in methods],
        "Estimate":   [mc_est[(str(k), m)] for k in strikes for m in methods],
        "S.E.":       [se_vals[(str(k), m)] for k in strikes for m in methods],
        "R.E.":       [re_vals[(str(k), m)] for k in strikes for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T

    return df

In [3]:
results_7_1 = example_7_1()
results_7_1

C:\Users\ningc\AppData\Local\Temp\ipykernel_11792\448699965.py:33: RuntimeWarning: invalid value encountered in scalar divide
  re = se / np.abs(est)


Strike price       50                60                70                80  \
                   IS    plain       IS    plain       IS    plain       IS   
True value   5.15e-01          3.77e-02          4.54e-04          1.64e-06   
Estimate     5.23e-01 5.21e-01 3.74e-02 3.65e-02 4.63e-04 5.94e-04 1.64e-06   
S.E.         5.15e-03 4.94e-03 5.45e-04 1.87e-03 8.86e-06 2.42e-04 3.78e-08   
R.E.         9.85e-03 9.49e-03 1.46e-02 5.11e-02 1.91e-02 4.08e-01 2.30e-02   

Strike price           
                plain  
True value             
Estimate     0.00e+00  
S.E.         0.00e+00  
R.E.              NaN

**Example 7.2.** The setup is the same as Example 7.1. Use importance sampling to estimate the price of a call option with maturity $T$ and strike price $K$. Compare with the plain Monte Carlo scheme.

In [4]:
def example_7_2():
    np.random.seed(4198)
    
    # simulation params
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    n = 10_000
    
    def importance_sampling(K):
        def h(x):
            Y = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * x)
            return np.exp(-r * T) * np.maximum(Y - K, 0)
        
        def hf_grad(x):
            a = sig * np.sqrt(T)
            b = S_0 * np.exp(-0.5 * sig**2 * T + a * x) * (a - x)
            return b + np.exp(-r * T) * K * x
    
        x_star = bisection_method(hf_grad, 0, 5)

        Y = np.random.normal(loc=x_star, size=n)
        H = h(Y) * np.exp(- x_star * Y + x_star**2 * 0.5)

        # compute mean est. and std. error
        est = np.mean(H)
        se = sem(H)
        re = se / np.abs(est)

        return est, se, re, x_star
    
    def plain_sampling(K):
        # sampling
        Z = np.random.normal(size=n)
        Y = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * Z)
        X = np.exp(-r * T) * np.maximum(Y - K, 0)

        # compute mean est. and std. error
        est = np.mean(X)
        se = sem(X)
        re = se / np.abs(est)

        return est, se, re

    theoretical = {}
    mc_est = {}
    se_vals = {}
    re_vals = {}
    x_stars = {}
    strikes, methods = [50, 60, 70, 80, 100, 120], ["IS", "plain"]
    
    for k in strikes:
        theoretical_val = european_call(S_0, k, r, T, sig) 
        mc_est_plain, se_plain, re_plain = plain_sampling(k)
        mc_est_importance, se_importance, re_importance, x_star = importance_sampling(k)

        re_importance = f"{100 * re_importance:.2f}%"
        re_plain = f"{100 * re_plain:.2f}%"

        importance_key = (f"{k}", "IS")
        plain_key = (f"{k}", "plain")

        theoretical[importance_key] = theoretical_val
        mc_est[importance_key] = mc_est_importance
        se_vals[importance_key] = se_importance
        re_vals[importance_key] = re_importance
        x_stars[importance_key] = x_star

        theoretical[plain_key] = ""
        mc_est[plain_key] = mc_est_plain
        se_vals[plain_key] = se_plain
        re_vals[plain_key] = re_plain
        x_stars[plain_key] = ""
    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [strikes, methods],
        names=["Strike price", ""]
    )

    # Collect rows
    rows = {
        "True value": [theoretical[(str(k), m)] for k in strikes for m in methods],
        "Estimate":   [mc_est[(str(k), m)] for k in strikes for m in methods],
        "S.E.":       [se_vals[(str(k), m)] for k in strikes for m in methods],
        "R.E.":       [re_vals[(str(k), m)] for k in strikes for m in methods],
        "x*":         [x_stars[(str(k), m)] for k in strikes for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T

    return df

In [5]:
results_7_2 = example_7_2()
results_7_2

C:\Users\ningc\AppData\Local\Temp\ipykernel_11792\1824503156.py:39: RuntimeWarning: invalid value encountered in scalar divide
  re = se / np.abs(est)


Strike price      50                60                70                80   \
                   IS    plain       IS    plain       IS    plain       IS   
True value   5.23e+00          1.62e+00          3.92e-01          7.95e-02   
Estimate     5.28e+00 5.25e+00 1.62e+00 1.55e+00 3.97e-01 3.94e-01 7.89e-02   
S.E.         2.46e-02 7.29e-02 1.11e-02 4.17e-02 3.39e-03 2.12e-02 8.05e-04   
R.E.            0.47%    1.39%    0.69%    2.69%    0.85%    5.39%    1.02%   
x*           9.85e-01          1.48e+00          2.05e+00          2.60e+00   

Strike price               100               120           
                plain       IS    plain       IS    plain  
True value            2.40e-03          6.07e-05           
Estimate     6.73e-02 2.41e-03 3.83e-03 6.13e-05 0.00e+00  
S.E.         8.64e-03 3.00e-05 1.78e-03 8.74e-07 0.00e+00  
R.E.           12.84%    1.24%   46.40%    1.43%     nan%  
x*                    3.60e+00          4.46e+00

**Example 7.3.** Consider a much simplified credit risk model with m independent obligors. Denote by $p_k$ the probability that the $k$-th obligor defaults
and $c_k$ the loss resulting from its default. Assuming that $c_k = 1$ for every $k$,
the total loss is
$$ L = \sum_{k = 1}^{m} c_k X_k = \sum_{k = 1}^{m} X_k, $$
where $X_k$ is the default indicator for the $k$-th obligor, that is, $\{X_k\}$ are independent Bernoulli random variables such that
$$\mathbb{P}(X_k = 1) = p_k, \quad  \mathbb{P}(X_k = 0) = 1 − p_k.$$
Assuming that x is a large threshold, use importance sampling to estimate the tail probability $\mathbb{P}(L > x)$.

In [6]:
def example_7_3():
    np.random.seed(4198)
    
    # simulation params
    m = 1_000
    k = np.arange(1, m + 1)
    p = 0.01 * (1.0 + np.exp(-k / m))
    n = 10_000
    
    def importance_sampling(x):
        def p_bar(theta):
            return p * np.exp(theta) / (1.0 + p * (np.exp(theta) - 1))
        
        def grad(theta):
            return np.sum(p_bar(theta)) - x
    
        theta_opt = bisection_method(grad, 0, 10)
        p_bar_opt = p_bar(theta_opt)

        Y = bernoulli.rvs(p=p_bar_opt, size=(n, m))
        L_tilted = Y.sum(axis=1)
        phi_opt = np.sum(np.log(1 + p * (np.exp(theta_opt) - 1)))

        H = np.exp(- theta_opt * L_tilted + phi_opt) * (L_tilted > x)

        # compute mean est. and std. error
        est = np.mean(H)
        se = sem(H)
        re = se / np.abs(est)

        return est, se, re, theta_opt

    def plain_sampling(x):
        # sampling
        X = bernoulli.rvs(p=p, size=(n, m))
        L = np.sum(X, axis=1)
        H = L > x

        # compute mean est. and std. error
        est = np.mean(H)
        se = sem(H)
        re = se / np.abs(est)

        return est, se, re

    mc_est = {}
    se_vals = {}
    re_vals = {}
    theta_opts = {}
    x, methods = [20, 30, 40], ["IS", "plain"]
    
    for xx in x: 
        mc_est_plain, se_plain, re_plain = plain_sampling(xx)
        mc_est_importance, se_importance, re_importance, theta_opt = importance_sampling(xx)

        re_importance = f"{100 * re_importance:.2f}%"
        re_plain = f"{100 * re_plain:.2f}%"

        importance_key = (f"{xx}", "IS")
        plain_key = (f"{xx}", "plain")

        mc_est[importance_key] = mc_est_importance
        se_vals[importance_key] = se_importance
        re_vals[importance_key] = re_importance
        theta_opts[importance_key] = theta_opt

        mc_est[plain_key] = mc_est_plain
        se_vals[plain_key] = se_plain
        re_vals[plain_key] = re_plain
        theta_opts[plain_key] = ""
    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [x, methods],
        names=["x ", ""]
    )

    # Collect rows
    rows = {
        "Estimate":   [mc_est[(str(xx), m)] for xx in x for m in methods],
        "S.E.":       [se_vals[(str(xx), m)] for xx in x for m in methods],
        "R.E.":       [re_vals[(str(xx), m)] for xx in x for m in methods],
        "θ*":         [theta_opts[(str(xx), m)] for xx in x for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T

    return df

In [7]:
results_7_3 = example_7_3()
results_7_3

C:\Users\ningc\AppData\Local\Temp\ipykernel_11792\2362098857.py:42: RuntimeWarning: invalid value encountered in scalar divide
  re = se / np.abs(est)


x              20                30                40         
               IS    plain       IS    plain       IS    plain
Estimate 1.48e-01 1.51e-01 6.93e-04 2.00e-04 1.60e-07 0.00e+00
S.E.     1.93e-03 3.58e-03 1.38e-05 1.41e-04 3.95e-09 0.00e+00
R.E.        1.31%    2.37%    1.99%   70.71%    2.47%     nan%
θ*       2.07e-01          6.23e-01          9.21e-01

### Cross Entropy Method
#### The Basic Cross-Entropy Algorithm

**Example 7.4.** Assume that under the risk-neutral probability measure, the underlying stock price is a geometric Brownian motion
$$ S_t = S_0 \exp \left\{ \left( r - \frac{1}{2} \sigma^2 \right) t + \sigma W_t \right\} ,$$
where $r$ is the risk-free interest rate. Estimate the price of a call option with strike price $K$ and maturity $T$. Compare with Example 7.2.

In [8]:
def example_7_4():
    np.random.seed(4198)
    
    # simulation params
    N = 2_000
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    n = 10_000
    
    def cross_entropy(K):
        def h(x):
            X = S_0 * np.exp(-0.5 * sig**2 * T + sig * np.sqrt(T) * x)
            return np.maximum(X - np.exp(-r * T) * K, 0)
        
        theta_hat = norm_cross_entropy(h, 1, N)

        Y = np.random.normal(loc=theta_hat, size=n)
        H = h(Y) * np.exp(-theta_hat * Y + theta_hat**2 * 0.5)

        # compute mean est. and std. error
        est = np.mean(H)
        se = sem(H)
        re = se / np.abs(est)

        return est, se, re, theta_hat
    
    def basic_importance_sampling(K):
        def h(x):
            Y = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * x)
            return np.exp(-r * T) * np.maximum(Y - K, 0)
        
        def hf_grad(x):
            a = sig * np.sqrt(T)
            b = S_0 * np.exp(-0.5 * sig**2 * T + a * x) * (a - x)
            return b + np.exp(-r * T) * K * x
    
        x_star = bisection_method(hf_grad, 0, 5)

        Y = np.random.normal(loc=x_star, size=n)
        H = h(Y) * np.exp(-x_star * Y + x_star**2 * 0.5)

        # compute mean est. and std. error
        est = np.mean(H)
        se = sem(H)
        re = se / np.abs(est)

        return est, se, re, x_star
        

    true_vals = {}
    mc_est = {}
    se_vals = {}
    re_vals = {}
    theta_opts = {}
    K, methods = [50, 60, 80, 100, 120], ["basic", "ce"]
    
    for kk in K:
        true_val = european_call(S_0, kk, r, T, sig)
        mc_est_basic, se_basic, re_basic, theta_opt_basic = basic_importance_sampling(kk)
        mc_est_ce, se_ce, re_ce, theta_opt_ce = cross_entropy(kk)

        re_basic = f"{100 * re_basic:.2f}%"
        re_ce = f"{100 * re_ce:.2f}%"

        basic_key = (f"{kk}", "basic")
        ce_key = (f"{kk}", "ce")

        true_vals[basic_key] = ""
        mc_est[basic_key] = mc_est_basic
        se_vals[basic_key] = se_basic
        re_vals[basic_key] = re_basic
        theta_opts[basic_key] = theta_opt_basic

        true_vals[ce_key] = true_val
        mc_est[ce_key] = mc_est_ce
        se_vals[ce_key] = se_ce
        re_vals[ce_key] = re_ce
        theta_opts[ce_key] = theta_opt_ce
    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [K, methods],
        names=["K ", ""]
    )

    # Collect rows
    rows = {
        "True value": [true_vals[(str(kk), m)] for kk in K for m in methods],
        "Estimate":   [mc_est[(str(kk), m)] for kk in K for m in methods],
        "S.E.":       [se_vals[(str(kk), m)] for kk in K for m in methods],
        "R.E.":       [re_vals[(str(kk), m)] for kk in K for m in methods],
        "θ*":         [theta_opts[(str(kk), m)] for kk in K for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T

    return df

In [9]:
results_7_4 = example_7_4()
results_7_4

c:\Users\ningc\Documents\university\bachelor-4\semester-1\capstone\project\src\utility.py:53: RuntimeWarning: invalid value encountered in divide
  theta_hat = np.sum(hX.reshape(-1, 1) * X, axis=0) / np.sum(hX)


K               50                60                80                100  \
              basic       ce    basic       ce    basic       ce    basic   
True value          5.23e+00          1.62e+00          7.95e-02            
Estimate   5.26e+00 5.23e+00 1.62e+00 1.62e+00 8.01e-02 7.94e-02 2.41e-03   
S.E.       2.47e-02 2.46e-02 1.12e-02 1.08e-02 8.09e-04 8.02e-04 3.03e-05   
R.E.          0.47%    0.47%    0.69%    0.67%    1.01%    1.01%    1.26%   
θ*         9.85e-01 1.27e+00 1.48e+00 1.74e+00 2.60e+00 2.87e+00 3.60e+00   

K                        120           
                 ce    basic       ce  
True value 2.40e-03          6.07e-05  
Estimate        NaN 6.02e-05      NaN  
S.E.            NaN 8.61e-07      NaN  
R.E.           nan%    1.43%     nan%  
θ*              NaN 4.46e+00      NaN

**Example 7.5.** Consider a discretely monitored average price call option
with payoff $( \bar{S} - K)^+$ and maturity $T$. Here $\bar{S}$ is the arithmetic mean
$$ \bar{S} \sum_{k = 1}^{m} S_{t_k} $$
for a given set of dates $0 < t_1 < ··· < t_m = T$. Assume that under the risk-neutral probability measure the price of the underlying asset is a geometric Brownian motion
$$ S_t = S_0 \exp \left\{ \left( r - \frac{1}{2} \sigma^2 \right) t + \sigma W_t \right\} ,$$
where $r$ is the risk-free interest rate. Estimate the option price.

In [10]:
def example_7_5():
    np.random.seed(4198)

    # simulation parameters
    N = 2_000
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    m = 12
    n = 10_000
    dt = T / 12

    t = np.arange(1, m + 1, dtype=float) * dt
    sqrt_dt = np.sqrt(dt)

    def cross_entropy(K):
        def h(X):
            sum_Z = np.cumsum(X, axis=0)
            S_t = S_0 * np.exp((r - 0.5 * sig**2) * t + sig * sqrt_dt * sum_Z)
            S_bar = S_t.mean()
            return np.exp(-r * T) * np.maximum(S_bar - K, 0)
    
        theta_hat = norm_cross_entropy(h, m, N)
        Y = multivariate_normal.rvs(mean=theta_hat, size=n)
        sum_Y = np.cumsum(Y, axis=1)

        S_t = S_0 * np.exp((r - 0.5 * sig**2) * t + sig * sqrt_dt * sum_Y)
        S_bar = S_t.mean(axis=1)

        tilt = np.exp(-np.sum(theta_hat * Y, axis=1) + 0.5 * np.sum(theta_hat**2))
        H = np.exp(-r * T) * np.maximum(S_bar - K, 0) * tilt

        # compute mean est. and std. err
        est = np.mean(H)
        se = sem(H)
        re = se / np.abs(est)

        return est, se, re
    
    def plain(K):
        # sampling
        Z = np.random.normal(size=(n, m))
        sum_Z = np.cumsum(Z, axis=1)

        S_t = S_0 * np.exp((r - 0.5 * sig**2) * t + sig * sqrt_dt * sum_Z)
        S_bar = S_t.mean(axis=1)
        X = np.exp(-r * T) * np.maximum(S_bar - K, 0)

        # compute mean est. and std. err
        est = np.mean(X)
        se = sem(X)
        re = se / np.abs(est)

        return est, se, re
    
    mc_est = {}
    se_vals = {}
    re_vals = {}
    K, methods = [50, 60, 70], ["ce", "plain"]
    
    for kk in K:
        mc_est_plain, se_plain, re_plain = plain(kk)
        mc_est_ce, se_ce, re_ce = cross_entropy(kk)

        re_plain = f"{100 * re_plain:.2f}%"
        re_ce = f"{100 * re_ce:.2f}%"

        plain_key = (f"{kk}", "plain")
        ce_key = (f"{kk}", "ce")

        mc_est[plain_key] = mc_est_plain
        se_vals[plain_key] = se_plain
        re_vals[plain_key] = re_plain

        mc_est[ce_key] = mc_est_ce
        se_vals[ce_key] = se_ce
        re_vals[ce_key] = re_ce
    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [K, methods],
        names=["K ", ""]
    )

    # Collect rows
    rows = {
        "Estimate":   [mc_est[(str(kk), m)] for kk in K for m in methods],
        "S.E.":       [se_vals[(str(kk), m)] for kk in K for m in methods],
        "R.E.":       [re_vals[(str(kk), m)] for kk in K for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T

    return df
    

In [11]:
results_7_5 = example_7_5()
results_7_5

K              50                60                70         
               ce    plain       ce    plain       ce    plain
Estimate 3.09e+00 3.11e+00 3.34e-01 3.31e-01 1.67e-02 1.80e-02
S.E.     1.51e-02 4.32e-02 3.07e-03 1.47e-02 7.24e-04 2.93e-03
R.E.        0.49%    1.39%    0.92%    4.45%    4.34%   16.27%

#### The General Iterative Cross-Entropy Algorithm

**Example 7.6.** Let us revisit the problem of estimating the call option price in Example 7.4. The basic cross-entropy method fails when the strike price $K$ is exceedingly large. Design an iterative cross-entropy scheme to resolve this issue.

In [ ]:
def example_7_6():
    np.random.seed(4198)
    
    # simulation params
    N = 2_000
    it_num = 5
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    n = 10_000
    
    def iterative_cross_entropy(K):
        def h(x):
            X = S_0 * np.exp(-0.5 * sig**2 * T + sig * np.sqrt(T) * x)
            return np.maximum(X - np.exp(-r * T) * K, 0)
        
        theta_hat_0 = np.log(K / S_0) / (sig * np.sqrt(T)) - r / sig * np.sqrt(T)
        theta_hat = theta_hat_0
        theta_hist = [theta_hat_0]
        for _ in range(it_num):
            theta_hat = iterative_norm_cross_entropy(h, 1, N, theta_hat)
            theta_hist.append(theta_hat)

        Y = np.random.normal(loc=theta_hat, size=n)
        tilt = np.exp(-theta_hat * Y + theta_hat**2 * 0.5)
        H = h(Y) * tilt

        # compute mean est. and std. error
        est = np.mean(H)
        se = sem(H)
        re = se / np.abs(est)

        return est, se, re, theta_hat, theta_hist
    

    true_vals = {}
    mc_est = {}
    se_vals = {}
    re_vals = {}
    theta_opts = {}
    theta_hists = {}
    K = [50, 60, 80, 100, 120]
    
    for kk in K:
        true_val = european_call(S_0, kk, r, T, sig)
        mc_est_ce, se_ce, re_ce, theta_opt_ce, theta_hist_ce = iterative_cross_entropy(kk)

        re_ce = f"{100 * re_ce:.2f}%"

        ice_key = (f"{kk}")

        true_vals[ice_key] = true_val
        mc_est[ice_key] = mc_est_ce
        se_vals[ice_key] = se_ce
        re_vals[ice_key] = re_ce
        theta_opts[ice_key] = theta_opt_ce
        theta_hists[ice_key] = theta_hist_ce

    
    # Build MultiIndex columns
    columns1 = pd.MultiIndex.from_product([K], names=["K "])
    columns2 = pd.MultiIndex.from_product([[f"i={i}" for i in range(it_num + 1)]], names=["theta_i "])

    # Collect rows
    rows1 = {
        "True value": [true_vals[str(kk)] for kk in K],
        "Estimate":   [mc_est[str(kk)] for kk in K],
        "S.E.":       [se_vals[str(kk)] for kk in K],
        "R.E.":       [re_vals[str(kk)] for kk in K],
        "θ*":         [theta_opts[str(kk)] for kk in K],
    }

    rows2 = {
        f"K={kk}" : theta_hists[(str(kk))] for kk in K
    }

    # Create DataFrame
    df1 = pd.DataFrame(rows1, index=columns1).T
    df2 = pd.DataFrame(rows2, index=columns2).T

    return df1, df2

In [13]:
results_7_6 = example_7_6()
df1, df2 = results_7_6

display(df1)
display(df2)

K,50,60,80,100,120
True value,5.23e+00,1.62e+00,7.95e-02,2.40e-03,6.07e-05
Estimate,5.24e+00,1.62e+00,8.00e-02,2.38e-03,6.03e-05
S.E.,2.42e-02,1.09e-02,8.00e-04,2.99e-05,8.60e-07
R.E.,0.46%,0.67%,1.00%,1.25%,1.43%
θ*,1.22e+00,1.76e+00,2.86e+00,3.82e+00,4.65e+00


theta_i,i=0,i=1,i=2,i=3,i=4,i=5
K=50,-2.50e-01,1.21e+00,1.24e+00,1.22e+00,1.21e+00,1.22e+00
K=60,6.62e-01,1.76e+00,1.78e+00,1.76e+00,1.77e+00,1.76e+00
K=80,2.10e+00,2.86e+00,2.86e+00,2.87e+00,2.86e+00,2.86e+00
K=100,3.22e+00,3.82e+00,3.82e+00,3.83e+00,3.82e+00,3.82e+00
K=120,4.13e+00,4.65e+00,4.65e+00,4.66e+00,4.65e+00,4.65e+00


**Example 7.7.** Revisit Example 7.5 for the pricing of average price call options. When the strike price $K$ gets larger, the plain Monte Carlo scheme or the basic cross-entropy method will start to fail. Design an iterative cross-entropy scheme to estimate the option price.

In [14]:
def example_7_7():
    np.random.seed(4198)
    
    # simulation parameters
    N = 2_000
    it_num = 5
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    n, m = 10_000, 12

    dt = T / m
    sqrt_dt = np.sqrt(dt)
    t = np.linspace(dt, T, m)
    
    def iterative_cross_entropy(K):
        def f(a):
            St = S_0 * np.exp(r * t + a * sig * t / sqrt_dt) 
            return St.mean() - K

        def h(X):
            sum_Z = np.cumsum(X, axis=0)
            S_t = S_0 * np.exp((r - 0.5 * sig**2) * t + sig * sqrt_dt * sum_Z)
            S_bar = S_t.mean()
            return np.exp(-r * T) * np.maximum(S_bar - K, 0)

        a_opt = bisection_method(f, -10, 5)
        theta_hat_0 = np.repeat(a_opt, m)
        theta_hat = theta_hat_0
        theta_hist = [theta_hat_0]
        for _ in range(it_num):
            theta_hat = iterative_norm_cross_entropy(h, m, N, theta_hat)
            theta_hist.append(theta_hat)

        Y = multivariate_normal.rvs(mean=theta_hat, size=n)
        sum_Y = np.cumsum(Y, axis=1)

        S_t = S_0 * np.exp((r - 0.5 * sig**2) * t + sig * sqrt_dt * sum_Y)
        S_bar = S_t.mean(axis=1)

        tilt = np.exp(- np.sum(theta_hat * Y, axis=1) + 0.5 * np.sum(theta_hat**2))
        H = np.exp(-r * T) * np.maximum(S_bar - K, 0) * tilt
    
        # compute mean est. and std. error
        est = np.mean(H)
        se = sem(H)
        re = se / np.abs(est)

        return est, se, re, theta_hat, theta_hist
    

    true_vals = {}
    mc_est = {}
    se_vals = {}
    re_vals = {}
    theta_opts = {}
    theta_hists = {}
    K = [50, 60, 70, 80, 100]
    
    for kk in K:
        true_val = european_call(S_0, kk, r, T, sig)
        mc_est_ce, se_ce, re_ce, theta_opt_ce, theta_hist_ce = iterative_cross_entropy(kk)

        re_ce = f"{100 * re_ce:.2f}%"

        ice_key = (f"{kk}")

        true_vals[ice_key] = true_val
        mc_est[ice_key] = mc_est_ce
        se_vals[ice_key] = se_ce
        re_vals[ice_key] = re_ce
        theta_opts[ice_key] = theta_opt_ce
        theta_hists[ice_key] = theta_hist_ce

    
    # Build MultiIndex columns
    columns1 = pd.MultiIndex.from_product([K], names=["K "])

    # Collect rows
    rows1 = {
        "Estimate":   [mc_est[str(kk)] for kk in K],
        "S.E.":       [se_vals[str(kk)] for kk in K],
        "R.E.":       [re_vals[str(kk)] for kk in K],
    }

    # Create DataFrame
    df1 = pd.DataFrame(rows1, index=columns1).T

    return df1

In [15]:
results_7_7 = example_7_7()
results_7_7

K,50,60,70,80,100
Estimate,3.09e+00,3.34e-01,1.66e-02,4.93e-04,2.11e-07
S.E.,1.47e-02,2.78e-03,1.83e-04,6.60e-06,3.45e-09
R.E.,0.48%,0.83%,1.10%,1.34%,1.63%


**Example 7.8.** Consider a multi-asset basket call option with maturity $T$ and payoff
$$ \left( c_1 S_T^{(1)}  + \cdots + c_d S_T^{(d)} - K \right) ^+.$$
Under the risk-neutral probability measure, the underlying stock prices are assumed to be geometric Brownian motions:
$$ S_t^{(i)} = S_0^{(i)} \exp \left\{ \left( r - \frac{1}{2} \sigma_i^2 \right) t + \sigma_i W_t^{(i)} \right\}, \quad d = 1, 2, ..., d,$$
where $W = (W^{(1)}, ..., W^{(d)})$ is a $d$-dimensional Brownian motion with covariance matrix $\Sigma = [\Sigma_{ij}]$ such that $\Sigma_{ii} = 1$ for all $i$. Use the cross-entropy method to estimate the option price.

In [16]:
def example_7_8():
    np.random.seed(4198)

    # Simulation Parameters
    d = 4
    S_0 = np.array([45, 50, 50, 55])
    r, T = 0.03, 0.5
    c = np.array([0.4, 0.2, 0.2, 0.2])
    sig = np.array([0.1, 0.1, 0.2, 0.2])
    cov = np.array([
        [1.0, 0.5, -0.3, 0.4],
        [0.5, 1.0, 0.3, 0.5],
        [-0.3, 0.3, 1.0, 0.7],
        [0.4, 0.5, 0.7, 1.0]
    ])
    A = np.linalg.cholesky(cov)

    n = 10_000
    it_num,  N = 5, 2_000

    def iterative_cross_entropy(K):
        def f(a):
            S = S_0 * np.exp(r * T + sig * np.sqrt(T) * a)
            S_bar = S @ c
            return S_bar - K
        
        def h(X):
            y = X @ A.T
            S_T = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * y)
            S_bar = S_T @ c
            return np.exp(-r * T) * np.maximum(S_bar - K, 0)
        
        a_opt = bisection_method(f, -20, 10)
        eta = np.repeat(a_opt, d)
        theta_hat_0 = np.linalg.solve(A, eta)

        theta_hat = theta_hat_0
        for _ in range(it_num):
            theta_hat = iterative_norm_cross_entropy(h, d, N, theta_hat)
        
        X = multivariate_normal.rvs(mean=theta_hat, size=n)
        H = h(X) * np.exp(-np.sum(theta_hat * X, axis=1) + 0.5 * np.sum(theta_hat**2))

        # compute mean est. and std. err
        est = np.mean(H)
        se = sem(H)
        re = se / np.abs(est)

        return est, se, re    

    
    def plain(K):
        X = np.random.normal(size=(n, d))
        W_T = np.sqrt(T) * X @ A.T
        S_T = S_0 * np.exp((r - 0.5 * sig**2) * T + sig * W_T)
        S_bar = S_T @ c
        Y = np.exp(-r * T) * np.maximum(S_bar - K, 0)

        # compute mean est. and std. err
        est = Y.mean()
        se = sem(Y)
        re = se / np.abs(est)
        
        return est, se, re
    
    mc_est = {}
    se_vals = {}
    re_vals = {}
    K, methods = [50, 60, 70], ["ce", "plain"]
    
    for kk in K:
        mc_est_plain, se_plain, re_plain = plain(kk)
        mc_est_ce, se_ce, re_ce = iterative_cross_entropy(kk)

        re_plain = f"{100 * re_plain:.2f}%"
        re_ce = f"{100 * re_ce:.2f}%"

        plain_key = (f"{kk}", "plain")
        ce_key = (f"{kk}", "ce")

        mc_est[plain_key] = mc_est_plain
        se_vals[plain_key] = se_plain
        re_vals[plain_key] = re_plain

        mc_est[ce_key] = mc_est_ce
        se_vals[ce_key] = se_ce
        re_vals[ce_key] = re_ce
    
    # Build MultiIndex columns
    columns = pd.MultiIndex.from_product(
        [K, methods],
        names=["K ", ""]
    )

    # Collect rows
    rows = {
        "Estimate":   [mc_est[(str(kk), m)] for kk in K for m in methods],
        "S.E.":       [se_vals[(str(kk), m)] for kk in K for m in methods],
        "R.E.":       [re_vals[(str(kk), m)] for kk in K for m in methods],
    }

    # Create DataFrame
    df = pd.DataFrame(rows, index=columns).T

    return df

In [17]:
results_7_8 = example_7_8()
results_7_8

C:\Users\ningc\AppData\Local\Temp\ipykernel_11792\3991271352.py:62: RuntimeWarning: invalid value encountered in scalar divide
  re = se / np.abs(est)


K              50                60                70         
               ce    plain       ce    plain       ce    plain
Estimate 1.31e+00 1.30e+00 7.98e-03 8.58e-03 3.99e-06 0.00e+00
S.E.     7.00e-03 2.10e-02 8.89e-05 1.63e-03 6.03e-08 0.00e+00
R.E.        0.54%    1.61%    1.11%   19.01%    1.51%     nan%

### Initiliazation in Rare Event Simulation
This section discusses a systematic way to find a good initial guess for the
cross-entropy method when the event of interest is rare.

**Example 7.9.** Let us revisit Example 7.6. The call option price can be written
as $v = E[H(X ; \alpha) 1_{{F(X \geq \alpha)}}]$, where $X$ is a standard normal
random variable and
$$ \alpha = K, \quad F(X) = S_0 \exp \left\{ \left( r - \frac{1}{2} \sigma^2 \right) T + \sigma \sqrt(T) X \right\}, $$
$$ H(X; \alpha) = e^{-r T} (F(X) - \alpha)^+. $$

In [18]:
def example_7_9():
    np.random.seed(4198)

    # Initilization parameters
    S_0, r, sig, T = 50, 0.05, 0.2, 1
    rho = 0.10
    N = 2000
    theta_0 = 0

    def intialize(K):
        alpha = K
        F = lambda X: S_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * X)
        H = lambda X, alpha: np.exp(-r * T) * np.maximum(F(X) - alpha, 0)

        N_0 = np.floor(N * (1 - rho))
        iter_num = 0
        theta_bar = theta_0

        # Write a helper function for initialization
        while True:
            iter_num += 1
            Y = np.random.normal(loc=theta_bar, size=N) # shape
            V = np.apply_along_axis(F, 0, Y)
            V_ord = np.sort(V)
            alpha_bar = V_ord[int(N_0)-1]
            
            HYa = H(Y, alpha_bar)
            Hya1 = HYa * (V >= alpha_bar)
            Hya1e = Hya1 * np.exp(-np.dot(theta_bar, Y)) # shape
            theta_bar = np.sum(Hya1e * Y, axis=0) / np.sum(Hya1e)

            if alpha_bar >= alpha: break
            if iter_num > 10: break

        return alpha_bar, theta_bar, iter_num
    
    iter_nums = {}
    alpha_bars = {}
    theta_hat_0s = {}
    strike_prices = [50, 60, 80, 100, 120]
    for K in strike_prices:
        alpha_bar, theta_hat_0, iter_num = intialize(K)
        key = f"K={K}"

        iter_nums[key] = iter_num
        alpha_bars[key] = alpha_bar
        theta_hat_0s[key] = theta_hat_0
    
    df = pd.DataFrame({
        "N_iter": [iter_nums[f"K={K}"] for K in strike_prices],
        "ᾱ": [alpha_bars[f"K={K}"] for K in strike_prices],
        "θ̂₀": [theta_hat_0s[f"K={K}"] for K in strike_prices],
    }, index=strike_prices)
    df.index.name = "K"
    return df.T

In [19]:
results_7_9 = example_7_9()

# force only N_iter to be displayed as an integer
pd.set_option("display.float_format", "{:.2f}".format)
results_7_9

K,50,60,80,100,120
N_iter,1.00,1.00,2.00,2.00,3.00
ᾱ,66.69,66.95,98.45,104.13,147.14
θ̂₀,2.16,2.14,3.73,4.01,5.61


### Applications to Risk Analysis
Use importance sampling to estimate the Value-at-Risk (VaR) of a portfolio loss,
where the tail probabilities involved are small.

#### Numerical Experiment

In [20]:
def numerical_experiment():
    np.random.seed(4198)

    m = 1_000
    n = 10_000
    N = 2_000
    it_num = 5

    k = np.arange(1, m+1, dtype=float)
    p = 0.01 * (1.0 + np.exp(- k / m))

    y = -norm.ppf(p)

    def apply_bisection_method(x):
        def helper(P):
            h = lambda theta: np.sum(P / (P + (1.0 - P) * np.exp(-theta))) - x
            return bisection_method(h, 0, 40) # unique positive root
        return helper

    def plain(x, rho):
        # Tilting Parameter
        mu = 0.0

        Z = np.random.normal(size=(n, 1))
        eps = np.random.normal(size=(n, m))

        Y = rho * Z + np.sqrt(1.0 - rho**2) * eps
        X = Y > y
        L = np.sum(X, axis=1)
        
        ind = L > x

        # Estimate Tail Probability
        tail_prob_est = np.mean(ind)
        tail_prob_se = sem(ind)
        tail_prob_re = tail_prob_se / np.abs(tail_prob_est)

        # Expected Tail Loss
        L1 = L * ind
        L1_est = np.mean(L1)

        etl = L1_est / tail_prob_est
        
        tl_se = np.sqrt(np.sum((L1 - etl * ind)**2)) / (n * tail_prob_est)
        tl_re = tl_se / np.abs(etl - x)

        return tail_prob_est, tail_prob_re, etl - x, tl_re, mu

    def importance_sampling(x, rho):
        # Initialize tilting parameter mu
        mu_hat = 0.0
        for _ in range(it_num):
            Z_bar = np.random.normal(loc=mu_hat, size=(N, 1))
            P = norm.cdf(- (y - rho * Z_bar) / np.sqrt(1 - rho**2)) # shape (N,m)

            l = np.sum(P, axis=1) # shape (N,)

            theta = np.zeros_like(l)
            theta[l < x] = np.apply_along_axis(apply_bisection_method(x), 1, P[l < x,:])
            
            exp_theta = np.exp(theta)[:, None] # shape (N,1)
            log_terms = np.log(1.0 + P * (exp_theta - 1.0))
            phi = np.sum(log_terms, axis=1)  # shape (N,)
            h_hat = np.exp(-theta * x + phi)   # shape (N,)

            # Update mu_hat
            Z_bar = Z_bar.flatten()
            mu_hat_Z_bar = mu_hat * Z_bar
            mu_hat = np.sum(h_hat * np.exp(-mu_hat_Z_bar) * Z_bar) / np.sum(h_hat * np.exp(-mu_hat_Z_bar))


        return _importance_sampling(x, rho, mu_hat=mu_hat)
    
    def partial_importance_sampling(x, rho):
        # Tilting parameter
        mu_hat = 0.0
        return _importance_sampling(x, rho, mu_hat)
            
    
    def _importance_sampling(x, rho, mu_hat=0):
        Z_bar = np.random.normal(loc=mu_hat, size=(n, 1))
        P = norm.cdf( -(y - rho * Z_bar) / np.sqrt(1 - rho**2)) # shape (n,m)
        l = np.sum(P, axis=1) # shape (n,)

        theta = np.zeros_like(l)
        theta[l < x] = np.apply_along_axis(apply_bisection_method(x), 1, P[l < x,:])
        exp_theta = np.exp(theta)[:, None] # shape (n,1)
        P_bar = P * (exp_theta / (1.0 + P * (exp_theta - 1.0))) # shape (n,m)

        X_bar = bernoulli.rvs(P_bar) # shape (n,m)
        L_bar = np.sum(X_bar, axis=1) # shape (n,)

        tilt = np.exp(-mu_hat * Z_bar.flatten() + mu_hat**2 * 0.5) # shape (n,)
        H = tilt * np.prod((P / P_bar)**X_bar * ((1 - P)/(1 - P_bar))**(1-X_bar), axis=1) # shape (n,)
        H = H * (L_bar > x)

        tail_prob_est = np.mean(H)
        tail_prob_se = sem(H)
        tail_prob_re = tail_prob_se / np.abs(tail_prob_est)

        # Expected Tail Loss
        etl = np.mean(L_bar * H) / tail_prob_est
        etl_se = np.sqrt(np.sum((L_bar * H - etl * H)**2)) / (n * tail_prob_est)
        etl_re = etl_se / np.abs(etl - x)

        return tail_prob_est, tail_prob_re, etl - x, etl_re, mu_hat
    
    x_rho_pairs = [
        (26, 0.05),
        (30, 0.1),
        (60, 0.3),
        (145, 0.6),
        (40, 0.1),
        (50, 0.1),
    ]

    dfs = []
    for x, rho in x_rho_pairs:
        # generate pandas dataframe for each (x, rho) pair
        # with rows: IS, Plain, Partial IS
        # and columns: TProb, TProb R.E., ETL - x, ETL R.E., mu_hat
        tail_prob_est = {}
        tail_prob_re = {}
        etl_minus_x = {}
        etl_re = {}
        mu_hats = {}

        methods = ["IS", "Plain MC", "Partial IS"]

        tp_est, tp_re, etl_x, etl_r, mu_hat = importance_sampling(x, rho)
        tail_prob_est["IS"] = tp_est
        tail_prob_re["IS"] = f"{100 * tp_re:.2f}%"
        etl_minus_x["IS"] = etl_x
        etl_re["IS"] = f"{100 * etl_r:.2f}%"
        mu_hats["IS"] = mu_hat

        tp_est, tp_re, etl_x, etl_r, mu_hat = plain(x, rho)
        tail_prob_est["Plain MC"] = tp_est
        tail_prob_re["Plain MC"] = f"{100 * tp_re:.2f}%"
        etl_minus_x["Plain MC"] = etl_x
        etl_re["Plain MC"] = f"{100 * etl_r:.2f}%"
        mu_hats["Plain MC"] = mu_hat

        tp_est, tp_re, etl_x, etl_r, mu_hat = partial_importance_sampling(x, rho)
        tail_prob_est["Partial IS"] = tp_est
        tail_prob_re["Partial IS"] = f"{100 * tp_re:.2f}%"
        etl_minus_x["Partial IS"] = etl_x
        etl_re["Partial IS"] = f"{100 * etl_r:.2f}%"
        mu_hats["Partial IS"] = mu_hat

        df = pd.DataFrame({
            "TProb": [tail_prob_est[m] for m in methods],
            "TProb R.E.": [tail_prob_re[m] for m in methods],
            "ETL - x": [etl_minus_x[m] for m in methods],
            "ETL R.E.": [etl_re[m] for m in methods],
            "μ̂": [mu_hats[m] for m in methods],
        }, index=methods)
        df.index.name = f"x={x}, ρ={rho}"
        dfs.append(df)
    
    return dfs

In [21]:
experiment_results = numerical_experiment()
for result in experiment_results:
    display(result)

C:\Users\ningc\AppData\Local\Temp\ipykernel_11792\538484154.py:36: RuntimeWarning: invalid value encountered in scalar divide
  tail_prob_re = tail_prob_se / np.abs(tail_prob_est)
C:\Users\ningc\AppData\Local\Temp\ipykernel_11792\538484154.py:42: RuntimeWarning: invalid value encountered in scalar divide
  etl = L1_est / tail_prob_est


,TProb,TProb R.E.,ETL - x,ETL R.E.,μ̂
"x=26, ρ=0.05",,,,,
IS,0.02,1.57%,2.60,1.08%,0.98
Plain MC,0.02,7.43%,2.90,5.24%,0.00
Partial IS,0.02,2.90%,2.57,2.01%,0.00


,TProb,TProb R.E.,ETL - x,ETL R.E.,μ̂
"x=30, ρ=0.1",,,,,
IS,0.02,1.52%,3.61,1.25%,1.62
Plain MC,0.02,7.92%,3.47,5.94%,0.00
Partial IS,0.02,5.15%,3.28,4.24%,0.00


,TProb,TProb R.E.,ETL - x,ETL R.E.,μ̂
"x=60, ρ=0.3",,,,,
IS,0.02,1.50%,16.01,1.27%,2.26
Plain MC,0.02,7.63%,15.58,6.77%,0.00
Partial IS,0.01,7.69%,16.05,8.28%,0.00


,TProb,TProb R.E.,ETL - x,ETL R.E.,μ̂
"x=145, ρ=0.6",,,,,
IS,0.02,1.54%,75.85,1.33%,2.44
Plain MC,0.01,8.65%,83.46,9.12%,0.00
Partial IS,0.02,7.83%,69.71,7.69%,0.00


,TProb,TProb R.E.,ETL - x,ETL R.E.,μ̂
"x=40, ρ=0.1",,,,,
IS,0.00,1.77%,3.30,1.29%,2.53
Plain MC,0.00,37.79%,2.43,20.13%,0.00
Partial IS,0.00,19.20%,4.02,14.45%,0.00


,TProb,TProb R.E.,ETL - x,ETL R.E.,μ̂
"x=50, ρ=0.1",,,,,
IS,0.00,1.95%,3.09,1.32%,3.36
Plain MC,0.00,nan%,NaN,nan%,0.00
Partial IS,0.00,59.19%,2.11,30.70%,0.00
